# Perseption
It's very important for robotics perception.Robotics can be broken down into three essential step cycle.
    Perception >> Dicision >> Action
This notebook related with Perception.For the task of perception we can use data from some kind of sensors like Lidor, Razor and Camera.

        Sensor    ||    Spartial Resolution    |    Dimension    |    Cost
    Lidor + Razor ||           Low             |       3D        |  Expensive
        Camera    ||           High            |       2D        |    Cheap

### Computer Vision
Computer vision is art and science to percieving and understanding the world around us through images.In self driving cars it helps us to detect lane marking, other cars and pedestrians and so on to navigate safely our cars.

## Finding Corners

## Correcting for Distortion

## Perspective Transform